In [1]:
import json
import urllib.request
import pandas as pd
import re
import random
import string
from spellchecker import SpellChecker
import numpy as np
from wordle_helpers import (
    words_with_commonly_used_letters,
    words_starting_with_commonly_used_letters,
    sorted_words_by_frequency,
    anagram_pairs,
    anagram_word_frequency,
    anagram_to_keep
)

import os
# where to save the data
ROOT_DIR = "."
FOLDER_NAME = "data"
PATH_TO_FOLDER = os.path.join(ROOT_DIR, FOLDER_NAME)
os.makedirs(PATH_TO_FOLDER, exist_ok=True)

# Load data  

I'll be using the Webster's Unabridged English Dictionary as my source of English words, helpfully curated [here](https://github.com/adambom/dictionary).

In [2]:
with urllib.request.urlopen("https://raw.githubusercontent.com/adambom/dictionary/master/dictionary.json") as url:
    data = json.load(url)
    
words = set([word.lower() for word in re.findall(r"[a-z]+", " ".join(list(data)), flags=re.IGNORECASE)])

In [3]:
list(words)[:10]

['godly',
 'paris',
 'populicide',
 'dermatoid',
 'anthracitic',
 'priestcraft',
 'motation',
 'pulmonated',
 'surfacer',
 'skyman']

In [4]:
len(words)

82702

# Select words 

In [5]:
new_words_list = words_starting_with_commonly_used_letters(words_with_commonly_used_letters(words))
print(new_words_list[:10], end="")

['paris', 'populicide', 'anthracitic', 'priestcraft', 'motation', 'pulmonated', 'surfacer', 'sanctified', 'cuneal', 'chronography']

In [6]:
def select_words(words_list, length_of_words=None):
    if length_of_words:
        return [word for word in words_list
                for length in length_of_words
                if len(word) == length]
    return list(words_list)

five_letter_words = select_words(new_words_list, length_of_words=[5])

In [7]:
print(five_letter_words[:10], end="")

['paris', 'patio', 'tosto', 'tasco', 'stood', 'iodal', 'cadge', 'stulp', 'acerb', 'tenon']

In [8]:
len(five_letter_words)

2809

## no repeat letter in 5 letter words  

So as to not waste letter guesses, we'll only select words with no repeat letters

In [9]:
no_repeat_letter_five_letter_words = [word for word in five_letter_words if len(set(word)) == 5]
print(no_repeat_letter_five_letter_words[:10], end="")

['paris', 'patio', 'tasco', 'iodal', 'cadge', 'stulp', 'acerb', 'slent', 'sowar', 'moted']

In [10]:
len(no_repeat_letter_five_letter_words)

1823

## Anagrams

In [11]:
# credit - https://twitter.com/mathsppblog/status/1547493884384710656?t=lWTpT5wZ3izeYRnNy9dc8g&s=03

def get_anagrams(words_list):
    """
    Returns a dict of word and anagrams. Anagrams are only for words with
    no repeat letters.
    """
    signatures = [sorted(word) for word in words_list]

    word_anagrams = {}
    #  iterate over signatures
    for signature in signatures:
        # get first word with this signature
        key_word = None
        for word in words_list:
            if sorted(word) == signature:
                key_word = word
                break
                
        anagram_matches = [word for sig, word in zip(signatures, words_list)
                           if sig == signature]
        word_anagrams[key_word] = [", ".join(anagram_matches)]
    return word_anagrams
    
anagrams_five_letter_words_no_repeat_letters = get_anagrams(no_repeat_letter_five_letter_words)

In [12]:
anagrams_five_letter_words_no_repeat_letters

{'paris': ['paris'],
 'patio': ['patio'],
 'tasco': ['tasco, costa, catso, scoat, coast'],
 'iodal': ['iodal'],
 'cadge': ['cadge, caged'],
 'stulp': ['stulp'],
 'acerb': ['acerb, caber'],
 'slent': ['slent'],
 'sowar': ['sowar'],
 'moted': ['moted'],
 'atimy': ['atimy, amity'],
 'impel': ['impel'],
 'shire': ['shire'],
 'flint': ['flint'],
 'thurl': ['thurl'],
 'punto': ['punto'],
 'pyrus': ['pyrus, pursy'],
 'since': ['since'],
 'coral': ['coral, claro, carol'],
 'sayer': ['sayer'],
 'sharp': ['sharp'],
 'smilt': ['smilt'],
 'calid': ['calid'],
 'metry': ['metry'],
 'tsung': ['tsung, stung'],
 'prest': ['prest'],
 'odist': ['odist'],
 'cubit': ['cubit'],
 'citer': ['citer, trice'],
 'scare': ['scare, sacre, serac, crase, carse'],
 'witan': ['witan, twain'],
 'ombre': ['ombre'],
 'canoe': ['canoe, ocean'],
 'arefy': ['arefy, faery, freya'],
 'comet': ['comet'],
 'facto': ['facto'],
 'cadis': ['cadis'],
 'stail': ['stail'],
 'flong': ['flong'],
 'flown': ['flown'],
 'shail': ['shail, a

In [13]:
# Save words as txt file. Text file provided in data folder
with open("./data/webster_dict_all_five_letter_words.txt", mode="w") as file:
    file.write("\n".join(five_letter_words))

# Determine anagrams to remove  

All anagrams in a group will share the same informational content, albeit different positional information. So where we find anagrams we select the one with the highest word frequency. If our guess word is an anagram of the challenge word we'll get yellows and greens.

## Get anagrams

In [14]:
# anagrams = np.concatenate([value for key, value in word_anagrams.items()])
anagrams = np.concatenate([value for value in anagrams_five_letter_words_no_repeat_letters.values()])

In [15]:
anagrams[:15]

array(['paris', 'patio', 'tasco, costa, catso, scoat, coast', 'iodal',
       'cadge, caged', 'stulp', 'acerb, caber', 'slent', 'sowar', 'moted',
       'atimy, amity', 'impel', 'shire', 'flint', 'thurl'], dtype='<U47')

In [16]:
anagrams.size

1418

In [17]:
anagrams_word_freq = anagram_word_frequency(anagrams)

In [18]:
anagrams_word_freq[:10]

array([list([('paris', 6.536702879786964e-07)]),
       list([('patio', 1.27347791534055e-06)]),
       list([('tasco', 0.0), ('costa', 4.2872547102395535e-07), ('catso', 0.0), ('scoat', 0.0), ('coast', 1.4112263812069209e-05)]),
       list([('iodal', 0.0)]),
       list([('cadge', 4.777053908447457e-08), ('caged', 1.408323867439763e-06)]),
       list([('stulp', 0.0)]),
       list([('acerb', 0.0), ('caber', 4.172363540289551e-08)]),
       list([('slent', 0.0)]), list([('sowar', 0.0)]),
       list([('moted', 0.0)])], dtype=object)

In [19]:
# save word_anagrams as csv
(pd.DataFrame(anagrams_five_letter_words_no_repeat_letters)
 .T
 .reset_index()
 .rename(columns={"index": "word", 0: "anagrams"})
).to_csv("./data/anagrams.csv", index=False)

## Keep anagrams with the highest frequency

In [20]:
keep_words = anagram_to_keep(anagrams_word_freq)
keep_words[:10]

array(['abhor', 'abide', 'abies', 'abler', 'abnet', 'abode', 'abord',
       'abort', 'about', 'absey'], dtype=object)

In [21]:
keep_words.size

1418

In [22]:
np.where(keep_words == "suine")

(array([1150]),)

In [23]:
# np.savetxt("./data/webster_common_starting_letters_final_words.txt", keep_words, fmt="%s")

# OR
with open("./data/webster_common_starting_letters_final_words.txt", mode="w") as file:
    file.write("\n".join(keep_words))